In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [4]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get('http://www.gims.go.kr/statistics.do?s_value=statistics_002')

# 테이블 버튼 누르기
xpath = """//*[@id="tab1"]/li[2]/a"""
time.sleep(2)
element_sel_gu = driver.find_element_by_xpath(xpath).click()

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [5]:
# 연도 xpath 뽑아오기
year_list_raw = driver.find_element_by_xpath("""//*[@id="select_year1"]""")
year_list = year_list_raw.find_elements_by_tag_name('option')

# 연도 리스트 뽑아오기
years_str = [option.get_attribute('value') for option in year_list]
years = []

for i in years_str:
    years.append(int(i))

# 지역 xpath 뽑아오기
loc_list_raw = driver.find_element_by_xpath("""//*[@id="select_sido1"]""")
loc_list = loc_list_raw.find_elements_by_tag_name('option')

# 지역 뽑아오기
locations = []
for i in range(0, len(loc_list)):
    locations.append(loc_list[i].text)

NameError: name 'driver' is not defined

In [6]:
usage_list = ['생활용수', '공업용수', '농업용수', '기타용수']
usage_df = pd.DataFrame(columns = years, index = usage_list)
usage_df

NameError: name 'years' is not defined

In [7]:
year_element = driver.find_element_by_id('select_year1')
usage_num = []

for i in range(0, 16):
    year_element.send_keys(years[i])
    
    time.sleep(2)
    
    try:
        WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                       'Timed out waiting for PA creation ' +
                                       'confirmation popup to appear.')

        alert = driver.switch_to.alert
        alert.accept()
        print("alert accepted")
    except TimeoutException:
        print("no alert")
    
    loc_element = driver.find_element_by_id('select_sido1')
    loc_element.send_keys(locations[-1])

    for i in range(2, 6):
        usage_num_xpath = driver.find_element_by_css_selector(f'#chart_table1 > tbody > tr:nth-child({i}) > td:nth-child(3)')
        # usage_tag = loc_list_raw.find_elements_by_tag_name('td')
        usage_num.append(usage_num_xpath.text)

NameError: name 'driver' is not defined

In [ ]:
usage_num

In [ ]:
usage_num_int = []
for i in usage_num:
    usage_num_str = i.replace(',', '')
    usage_num_int.append(int(float(usage_num_str)))

usage_num_int

In [ ]:
for i in range(0, int(len(usage_num_int)/4)):
    usage_df.iloc[0:, i] = usage_num_int[i*4:(i*4)+4]

usage_df

In [ ]:
usage_df = usage_df.T

In [ ]:
usage_df.to_csv("water_usage.csv", encoding="euc-kr")

In [ ]:
water = pd.read_csv('water.csv', encoding='euc-kr', index_col=0)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas_datareader.data as web
import matplotlib.pyplot as plt
%matplotlib inline

from fbprophet import Prophet
from datetime import datetime

In [ ]:
water

In [ ]:
pred_df = pd.DataFrame({'ds':water['연도'], 'y':water['생활용수']})
pred_df.reset_index(inplace=True)
pred_df